In [25]:
import os
import pandas as pd
import numpy as np

#import reportlab
from reportlab.pdfgen import canvas
from reportlab.lib import colors, utils
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle
from reportlab.lib.pagesizes import letter
from reportlab.platypus import SimpleDocTemplate, Table, TableStyle, Paragraph, Image
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet, ParagraphStyle
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

In [26]:
#Get current and parent directory directory
current_path = os.getcwd()
parent_directory = os.path.dirname(current_path)

#Path to the source folder where the raw data is located
source_path = os.path.join(parent_directory, 'data', 'raw', 'data_per_day.xlsx')

#Path to the image
image_path = os.path.join(parent_directory, 'graphics', 'bow.png')

#Path to the destination file
destination_path = os.path.join(parent_directory, 'data', 'processed', 'output.pdf')

#Read the .xlsx file
df = pd.read_excel(source_path, index_col=None)

In [27]:
#Main distinction is saved in the system as 'NA' and 'NNA'. 'NA' is read as a empty value.
#We need fill 'empty' value as a teachers, and change 'NNA' as a not teachers.
df['Grupa pracowników'] = df['Grupa pracowników'].fillna('Nauczyciel')
df['Grupa pracowników'] = df['Grupa pracowników'].replace('NNA', 'Nienauczyciel')

In [28]:
selected_columns = ['Pion'] #, 'Obszar kadrowy']

In [50]:
for col in selected_columns:
    for val in df[col].unique():
        
        
        
        df_pivot_mainGroups = df[df[col] == val].pivot_table(values='Etat', index='Grupa pracowników', columns=col, 
                              aggfunc=['count', 'sum']).applymap(lambda x: round(x, 2)) #.style.background_gradient(cmap='Greens')
        df_pivot_mainGroups = df_pivot_mainGroups.reset_index()
        df_pivot_mainGroups.columns = ['Grupa', 'Osoby', 'Etaty']
        
        
        
        df_pivot_teachersJobPosition = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                       'Etat', index='Stanowisko', columns=col, aggfunc=['count', 'sum']) \
                                       .applymap(lambda x: round(x, 2))
        df_pivot_teachersJobPosition = df_pivot_teachersJobPosition.reset_index()
        df_pivot_teachersJobPosition.columns = ['NA Stanowiska', 'Osoby', 'Etaty']
        
        
        
        df_pivot_teachersSubgroup = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                    'Etat', index='Podgrupa pracowników', columns=col, aggfunc=['count', 'sum']) \
                                    .applymap(lambda x: round(x, 2))
        df_pivot_teachersSubgroup.columns = df_pivot_teachersSubgroup.columns.droplevel(1)
        df_pivot_teachersSubgroup = df_pivot_teachersSubgroup.reset_index()
        df_pivot_teachersSubgroup.columns = ['NA Podgrupa', 'Osoby', 'Etaty']
                               
            
        
        df_pivot_notteachersSubgroup = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nienauczyciel')].pivot_table(values=
                                       'Etat', index='Podgrupa pracowników', columns=col, aggfunc=['count', 'sum']) \
                                       .applymap(lambda x: round(x, 2))
        df_pivot_notteachersSubgroup = df_pivot_notteachersSubgroup.reset_index()
        df_pivot_notteachersSubgroup.columns = ['NNA Grupa', 'Osoby', 'Średnia']
        
        
        df_teachersSalary = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nauczyciel')].pivot_table(values=
                                       'Wynagrodzenie', index=['Podgrupa pracowników', 'Stanowisko'], columns=col, aggfunc=['count', 'mean', 'median']) \
                                       .applymap(lambda x: round(x, 2))
        df_teachersSalary = df_teachersSalary.reset_index()
        df_teachersSalary.columns = ['NA Podgrupa','Stanowisko', 'Osoby', 'Średnia', 'Mediana']
        df_teachersSalary[['Średnia', 'Mediana']] = df_teachersSalary[['Średnia', 'Mediana']].apply(lambda col: col.apply(lambda x: '{:,.2f} zł'.format(x)))
        df_teachersSalary = df_teachersSalary.sort_values(by=['Stanowisko'])
        
        
        df_notteachersSalary = df[(df[col] == val) & (df['Grupa pracowników'] == 'Nienauczyciel')].pivot_table(values=
                                       'Wynagrodzenie', index=['Podgrupa pracowników'], columns=col, aggfunc=['count', 'mean', 'median']) \
                                       .applymap(lambda x: round(x, 2))
        df_notteachersSalary = df_notteachersSalary.reset_index()
        df_notteachersSalary.columns = ['NNA Podgrupa', 'Osoby', 'Średnia', 'Mediana']
        df_notteachersSalary[['Średnia', 'Mediana']] = df_notteachersSalary[['Średnia', 'Mediana']].apply(lambda col: col.apply(lambda x: '{:,.2f} zł'.format(x)))
        
        
        
        display(col, val, df_pivot_mainGroups, df_pivot_teachersJobPosition, df_pivot_teachersSubgroup, df_pivot_notteachersSubgroup, df_teachersSalary, df_notteachersSalary)

'Pion'

'Rektor'

,Grupa,Osoby,Etaty
0,Nauczyciel,942,839.48
1,Nienauczyciel,465,416.70


,NA Stanowiska,Osoby,Etaty
0,adiunkt,499,458.11
1,asystent,174,131.89
2,docent,3,3.00
3,profesor,103,91.98
4,profesor uczelni,128,124.90
5,starszy asystent,35,29.60


,NA Podgrupa,Osoby,Etaty
0,Badawczo-dydaktyczni,694,634.13
1,Badawczy,67,61.80
2,Dydaktyczni,181,143.55


,NNA Grupa,Osoby,Średnia
0,Administracyjno-ekon,208,200.60
1,Bibliotekarze,7,7.00
2,Informatycy,12,10.35
3,Inżynieryjno-techn.,117,85.70
4,Naukowo-badawczy,10,6.95
5,Naukowo-techniczni,13,8.60
6,Obsługa,85,84.50
7,Robotnicy,13,13.00


,NA Podgrupa,Stanowisko,Osoby,Średnia,Mediana
0,Badawczo-dydaktyczni,adiunkt,367,"7,043.83 zł","7,093.00 zł"
4,Badawczy,adiunkt,37,"6,839.14 zł","6,784.00 zł"
8,Dydaktyczni,adiunkt,95,"6,933.75 zł","6,962.00 zł"
1,Badawczo-dydaktyczni,asystent,117,"4,941.76 zł","4,921.00 zł"
5,Badawczy,asystent,25,"4,930.20 zł","4,775.00 zł"
9,Dydaktyczni,asystent,32,"5,209.66 zł","5,291.50 zł"
10,Dydaktyczni,docent,3,"6,660.33 zł","6,683.00 zł"
2,Badawczo-dydaktyczni,profesor,95,"12,444.25 zł","12,428.00 zł"
6,Badawczy,profesor,3,"13,020.33 zł","14,246.00 zł"
11,Dydaktyczni,profesor,5,"12,449.40 zł","12,901.00 zł"


,NNA Podgrupa,Osoby,Średnia,Mediana
0,Administracyjno-ekon,208,"6,331.96 zł","6,307.50 zł"
1,Bibliotekarze,7,"5,771.57 zł","5,553.00 zł"
2,Informatycy,12,"5,931.75 zł","6,061.00 zł"
3,Inżynieryjno-techn.,117,"6,063.58 zł","6,068.00 zł"
4,Naukowo-badawczy,10,"6,070.00 zł","6,217.50 zł"
5,Naukowo-techniczni,13,"6,188.08 zł","5,952.00 zł"
6,Obsługa,85,"4,720.21 zł","4,653.00 zł"
7,Robotnicy,13,"4,726.08 zł","4,733.00 zł"


In [30]:
# Tworzenie pliku PDF
c = canvas.Canvas(destination_path, pagesize=letter)
pdfmetrics.registerFont(TTFont('Calibri', 'calibri.ttf'))
pdfmetrics.registerFont(TTFont('Calibri-Bold', 'calibrib.ttf'))
c.setFont('Calibri', 20)  # Ustawienie używanego fontu

# Dodanie tekstu
c.drawString(235, 735, "HR - employee stats")

# Dodanie obrazu
img = utils.ImageReader(image_path)
c.drawImage(img, 205, 730, 25, 25)

#Dodanie jednego z pivot
styles = getSampleStyleSheet()
table_style = TableStyle([
    ('BACKGROUND', (0, 0), (-1, 0), colors.HexColor('#66CC00')),
    ('TEXTCOLOR', (0, 0), (-1, 0), colors.black),
    ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 0), (-1, 0), 'Calibri-Bold'),
    ('FONTSIZE', (0, 0), (-1, 0), 10),
    ('BOTTOMPADDING', (0, 0), (-1, 0), 4),
    ('BACKGROUND', (0, 1), (-1, -1), colors.white),
    ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
    ('ALIGN', (0, 1), (-1, -1), 'CENTER'),
    ('FONTNAME', (0, 1), (-1, -1), 'Calibri'),
    ('FONTSIZE', (0, 1), (-1, -1), 8),
    ('BOTTOMPADDING', (0, 1), (-1, -1), 2),
    ('GRID', (0, 0), (-1, -1), 1, colors.black)
])

header_row = df_pivot_notteachersSubgroup.columns.to_list()
table_data = [header_row] + df_pivot_notteachersSubgroup.values.tolist()

pdf_table = Table(table_data, style=table_style)

# Rysowanie tabeli na stronie PDF
pdf_table.wrapOn(c, 400, 600)  # 400 i 600 to szerokość i wysokość obszaru, w którym rysujemy tabelę
pdf_table.drawOn(c, 100, 500)  # 100 i 500 to współrzędne, gdzie rysujemy tabelę na stronie

# Zapisanie pliku PDF
c.save()